In [16]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [17]:
df = pd.read_csv("C:\\Users\\vishn\\Downloads\\extracted_data.csv")
df.head()

,time,HGT_prl,latitude,longitude,TMP_prl,TMP_2m,APCP_sfc,UGRD_10m,VGRD_10m
0,1990-01-01 00:00:00,5851.0586,13.68,80.16,289.79260,298.78370,1.409983e-14,-0.730697,0.115279
1,1990-01-01 06:00:00,5856.1550,13.68,80.16,288.84174,298.66785,2.255973e-13,-3.830207,-1.373627
2,1990-01-01 12:00:00,5853.1550,13.68,80.16,289.89853,299.04560,0.000000e+00,-4.528024,-1.926858
3,1990-01-01 18:00:00,5848.7900,13.68,80.16,288.81348,298.87085,0.000000e+00,-4.498979,-0.072557
4,1990-01-02 00:00:00,5851.9550,13.68,80.16,288.18890,299.14282,6.250000e-02,-2.182430,-1.781631


In [18]:
scaling_factor = 10**3  # Convert meters to millimeters
df['APCP_sfc'] = (df['APCP_sfc'] * scaling_factor).round().astype(int)

In [35]:
def compute_rmse(prediction, actual,  mean_dims = ('time')):
  error = prediction - actual
  rmse = np.sqrt(((error)**2 ).mean(mean_dims))
  return rmse
def compute_mae(prediction, actual, mean_dims = ('time')):
    error = prediction - actual
    mae = np.abs(error).mean(mean_dims)
    return mae
def compute_acc(prediction, actual):
    clim = actual.mean('time')
    try:
        t = np.intersect1d(prediction.time, actual.time)
        pred_anomaly = prediction.sel(time=t) - clim
    except AttributeError:
        t = actual.time.values
        pred_anomaly = prediction - clim
    act_anomaly = actual.sel(time=t) - clim

    pred_norm = pred_anomaly - pred_anomaly.mean()
    act_norm = act_anomaly - act_anomaly.mean()

    acc = (
            np.sum(pred_norm * act_norm) /
            np.sqrt(
                np.sum(pred_norm ** 2) * np.sum(act_norm ** 2)
            )
    )
    return acc
def compute_r_square(prediction, actual, mean_dims=('time')):
    # Calculate the mean of the actual values along the specified dimensions
    actual_mean = actual.mean(mean_dims)

    # Calculate the total sum of squares (SS_tot)
    ss_tot = ((actual - actual_mean) ** 2).sum(mean_dims)

    # Calculate the residual sum of squares (SS_res)
    ss_res = ((prediction - actual) ** 2).sum(mean_dims)

    # Calculate R-squared
    r_square = 1 - (ss_res / ss_tot)

    return r_square

In [55]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Filter the data by year (assuming 'time' column exists)
train_data = df[(df['time'] >= '1990') & (df['time'] < '2017')].drop(columns=['time'])
valid_data = df[(df['time'] >= '2017') & (df['time'] < '2019')].drop(columns=['time'])
test_data = df[(df['time'] >= '2019') & (df['time'] <= '2020')].drop(columns=['time'])

# Define features and target variables
#features = ['HGT_prl', 'TMP_2m', 'APCP_sfc', 'UGRD_10m', 'VGRD_10m']  # Example feature columns
features = ['UGRD_10m'] 
target = 'UGRD_10m'  # Example target column

# Split features and target variable
train_features = train_data[features]
train_target = train_data[target]
valid_features = valid_data[features]
valid_target = valid_data[target]
test_features = test_data[features]
test_target = test_data[target]

# Normalize the features using the mean and standard deviation of the training data
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
valid_features_scaled = scaler.transform(valid_features)
test_features_scaled = scaler.transform(test_features)

# Function to create windowed data
def create_windowed_data(features_data, target_data, past_steps, future_steps):
    X, y = [], []
    for i in range(len(features_data) - past_steps - future_steps + 1):
        X.append(features_data[i:i + past_steps])
        y.append(target_data[i + past_steps:i + past_steps + future_steps])
    return np.array(X), np.array(y)

# Parameters
past_steps = 28
future_steps = 8

# Create windowed data for each dataset
X_train, y_train = create_windowed_data(train_features_scaled, train_target.values, past_steps, future_steps)
X_valid, y_valid = create_windowed_data(valid_features_scaled, valid_target.values, past_steps, future_steps)
X_test, y_test = create_windowed_data(test_features_scaled, test_target.values, past_steps, future_steps)

# Now X_train, y_train, X_valid, y_valid, X_test, y_test are ready for model training and evaluation
print(f"Training data shape: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Validation data shape: X_valid={X_valid.shape}, y_valid={y_valid.shape}")
print(f"Test data shape: X_test={X_test.shape}, y_test={y_test.shape}")


Training data shape: X_train=(39413, 28, 1), y_train=(39413, 8)
Validation data shape: X_valid=(2885, 28, 1), y_valid=(2885, 8)
Test data shape: X_test=(1425, 28, 1), y_test=(1425, 8)


In [56]:
import tensorflow.keras as keras
from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, GRU, LSTMCell
from keras.layers import Conv2D, MaxPooling2D, Flatten, Bidirectional, LSTM, Dense, TimeDistributed, Conv1D, MaxPooling1D, Dropout, Conv3D
import warnings
warnings.filterwarnings('ignore')

In [57]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Define the number of features
num_features = X_train.shape[2]  # Number of features in your data

# Define the model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(past_steps, num_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
# Adjust the output layer to match the shape of y_train
model.add(Dense(future_steps)) # Output a single value for each time step

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_valid, y_valid))

# Make predictions
y_pred = model.predict(X_test)

# Reshape predictions to match y_test shape if necessary
# y_pred = y_pred.reshape(y_test.shape)

# Calculate R-squared value
from sklearn.metrics import r2_score
r2 = r2_score(y_test.flatten(), y_pred.flatten())

print(f'R-squared: {r2}')

# Print model summary
model.summary()

Epoch 1/10
1232/1232 [==============================] - 14s 11ms/step - loss: 7.1437 - val_loss: 6.2814
Epoch 2/10
1232/1232 [==============================] - 13s 11ms/step - loss: 6.8790 - val_loss: 6.0948
Epoch 3/10
1232/1232 [==============================] - 13s 10ms/step - loss: 4.3400 - val_loss: 3.1130
Epoch 4/10
1232/1232 [==============================] - 13s 10ms/step - loss: 3.5299 - val_loss: 2.9723
Epoch 5/10
1232/1232 [==============================] - 13s 11ms/step - loss: 3.3971 - val_loss: 2.9963
Epoch 6/10
1232/1232 [==============================] - 13s 11ms/step - loss: 3.2966 - val_loss: 2.9819
Epoch 7/10
1232/1232 [==============================] - 13s 11ms/step - loss: 3.2048 - val_loss: 3.0007
Epoch 8/10
1232/1232 [==============================] - 13s 11ms/step - loss: 3.1250 - val_loss: 3.0449
Epoch 9/10
1232/1232 [==============================] - 13s 11ms/step - loss: 3.0565 - val_loss: 3.0290
Epoch 10/10
45/45 [==============================] - 0s 3ms/step

In [58]:
import numpy as np
from sklearn.metrics import r2_score

def compute_rmse(prediction, actual):
    error = prediction - actual
    rmse = np.sqrt((error**2).mean())
    return rmse

def compute_mae(prediction, actual):
    error = prediction - actual
    mae = np.abs(error).mean()
    return mae

def compute_acc(prediction, actual):
    # Calculate climatology (mean of actual values)
    clim = actual.mean(axis=0)

    # Calculate anomalies (difference from the climatology)
    pred_anomaly = prediction - clim
    act_anomaly = actual - clim

    # Normalize the anomalies
    pred_norm = pred_anomaly - pred_anomaly.mean(axis=0)
    act_norm = act_anomaly - act_anomaly.mean(axis=0)

    # Calculate the Anomaly Correlation Coefficient (ACC)
    acc = (
        np.sum(pred_norm * act_norm) /
        np.sqrt(np.sum(pred_norm**2) * np.sum(act_norm**2))
    )
    return acc

def compute_r_square(prediction, actual):
    # Calculate R-squared using sklearn's r2_score
    r_square = r2_score(actual.flatten(), prediction.flatten())
    return r_square

# Make predictions
y_pred = model.predict(X_test)

# Compute metrics for the forecast period (output steps)
rmse = compute_rmse(y_pred, y_test)
mae = compute_mae(y_pred, y_test)
acc = compute_acc(y_pred, y_test)
r2 = compute_r_square(y_pred, y_test)

print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'ACC: {acc}')
print(f'R2: {r2}')


45/45 [==============================] - 0s 3ms/step
RMSE: 1.7817063899158627
MAE: 1.32574660294333
ACC: 0.7932782920623473
R2: 0.6058388943865008
